# **Chapter 14: Deep Computer Vision Using Convolutional Neural Networks**

## **1. Pendahuluan**

Meskipun komputer telah mengalahkan juara dunia catur pada tahun 1996, baru belakangan ini komputer mampu melakukan tugas yang tampak sepele bagi manusia, seperti mengenali anak anjing dalam gambar. Mengapa tugas ini begitu mudah bagi kita? Jawabannya terletak pada struktur otak kita: persepsi visual sebagian besar terjadi di luar kesadaran kita, diproses oleh modul visual khusus sebelum informasi tersebut mencapai kesadaran kita.

**Convolutional Neural Networks (CNN)** lahir dari studi tentang korteks visual otak ini. Berkat peningkatan daya komputasi dan data yang melimpah, CNN kini telah mencapai performa "superhuman" dalam beberapa tugas visual yang kompleks.

Dalam bab ini, kita akan membahas:
* Asal-usul CNN dan blok pembangunnya (Convolutional & Pooling Layers).
* Arsitektur CNN yang terkenal (LeNet, AlexNet, GoogLeNet, ResNet).
* Implementasi CNN menggunakan Keras dan TensorFlow.
* Konsep Object Detection (YOLO) dan Semantic Segmentation.


## **2. Arsitektur Korteks Visual**

Eksperimen Hubel dan Wiesel pada tahun 1950-an memberikan wawasan krusial tentang bagaimana kita melihat. Mereka menemukan bahwa neuron di korteks visual memiliki **Local Receptive Field** yang kecil, artinya mereka hanya bereaksi terhadap rangsangan visual di wilayah terbatas dari lapang pandang.

**Poin Penting:**
* Beberapa neuron bereaksi hanya terhadap garis horizontal, yang lain terhadap garis dengan orientasi berbeda.
* Neuron tingkat tinggi memiliki receptive field yang lebih besar dan bereaksi terhadap pola yang lebih kompleks yang merupakan kombinasi dari pola tingkat rendah.
* Struktur hierarkis ini memungkinkan deteksi pola kompleks di area mana pun dalam lapang pandang.

Ini menginspirasi arsitektur CNN: lapisan awal mendeteksi fitur sederhana (garis, tepi), dan lapisan berikutnya menggabungkannya menjadi fitur yang semakin kompleks.


## **3. Convolutional Layers**

Komponen terpenting dari CNN adalah **Convolutional Layer**. Berbeda dengan *fully connected layer* di mana setiap neuron terhubung ke semua piksel input, neuron di lapisan konvolusi hanya terhubung ke piksel dalam *receptive field*-nya.

### **Filter (Kernel)**
Bobot neuron dapat direpresentasikan sebagai gambar kecil seukuran receptive field, yang disebut **filter** atau **kernel** konvolusi.
* Misalnya, filter dengan garis vertikal putih di tengah akan mengabaikan segalanya kecuali garis vertikal di inputnya.
* Lapisan yang penuh dengan neuron yang menggunakan filter yang sama akan menghasilkan **Feature Map**, yang menyoroti area di gambar yang paling mengaktifkan filter tersebut.

### **Stride dan Padding**
* **Zero Padding:** Menambahkan nol di sekeliling input agar ukuran output tetap sama dengan input (disebut padding "SAME" di TensorFlow).
* **Stride:** Jarak langkah pergeseran antara satu receptive field ke receptive field berikutnya. Stride yang lebih besar mengurangi dimensi output secara drastis.

Mari kita lihat contoh penerapan filter secara manual menggunakan TensorFlow.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_sample_image

# Memuat gambar sampel
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

# Membuat 2 filter manual
filters = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)
filters[:, 3, :, 0] = 1  # Filter garis vertikal
filters[3, :, :, 1] = 1  # Filter garis horizontal

# Menerapkan filter ke gambar menggunakan tf.nn.conv2d
outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")

# Menampilkan Feature Map ke-2 (garis horizontal) dari gambar pertama
plt.imshow(outputs[0, :, :, 1], cmap="gray") 
plt.axis("off")
plt.title("Feature Map (Horizontal Filter)")
plt.show()


**Interpretasi Hasil:**
Kode di atas secara manual membuat filter garis vertikal dan horizontal, lalu menerapkannya pada gambar. Output yang ditampilkan adalah *feature map* yang menyoroti garis-garis horizontal pada gambar bangunan (seperti atap kuil). Bagian putih menunjukkan aktivasi tinggi (cocok dengan filter), sedangkan bagian gelap diabaikan. Dalam praktik nyata, CNN akan *belajar* sendiri filter mana yang paling berguna, tidak perlu didefinisikan manual.


## **4. Pooling Layers**

Tujuan **Pooling Layer** adalah untuk melakukan *subsample* (mengecilkan) gambar input guna mengurangi beban komputasi, penggunaan memori, dan jumlah parameter (mengurangi risiko overfitting).

### **Max Pooling**
Ini adalah jenis yang paling umum. Neuron pooling tidak memiliki bobot; ia hanya mengambil nilai maksimum dari input dalam receptive field-nya.
* Max pooling melestarikan fitur terkuat dan membuang sisanya.
* Ini memberikan sedikit invariansi terhadap translasi (pergeseran kecil pada gambar input tidak mengubah output pooling secara signifikan).

### **Global Average Pooling**
Sering digunakan di akhir CNN modern. Lapisan ini menghitung rata-rata dari *seluruh* feature map, menghasilkan satu angka per feature map.


In [ ]:
from tensorflow import keras

# Membuat model sederhana dengan Max Pooling
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()


## **5. Arsitektur CNN Populer**

Seiring waktu, arsitektur CNN berkembang menjadi lebih dalam dan kompleks. Kompetisi ImageNet (ILSVRC) menjadi tolok ukur utama kemajuan ini.

1. **LeNet-5 (1998):** Digunakan untuk pengenalan digit tulisan tangan (MNIST).
2. **AlexNet (2012):** Pemenang ILSVRC 2012 dengan margin besar.
3. **GoogLeNet (2014):** Memperkenalkan **Inception Module**.
4. **ResNet (2015):** Memperkenalkan **Skip Connections** untuk mengatasi *vanishing gradient*.

### **Implementasi ResNet-34**


In [ ]:
from tensorflow import keras

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, padding="same", use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides, padding="same", use_bias=False),
                keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[224, 224, 3], padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))

prev_filters = 64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

print("ResNet-34 berhasil dibangun.")


## **6. Menggunakan Pretrained Models (Transfer Learning)**

Biasanya kita tidak perlu melatih CNN dari nol. Kita bisa menggunakan model yang sudah dilatih pada dataset besar (seperti ImageNet) dan menggunakannya untuk tugas kita. Ini disebut **Transfer Learning**.

Berikut contoh memuat model **ResNet-50** yang sudah dilatih:


In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.applications.resnet50.ResNet50(weights="imagenet")

images_resized = tf.image.resize(images, [224, 224])
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)

Y_proba = model.predict(inputs)
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)

for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name}: {y_proba * 100:.2f}%")


**Pelajaran:**
Dengan hanya beberapa baris kode, kita bisa mendapatkan pengklasifikasi gambar yang sangat akurat. Model mendeteksi kelas 'palace' dan 'daisy' dengan probabilitas tinggi. Jika dataset kita kecil, kita bisa membekukan layer awal dan hanya melatih layer akhir model ini.


## **7. Object Detection & Semantic Segmentation**

Klasifikasi gambar hanya memberi tahu kita *apa* yang ada di gambar, bukan *di mana* letaknya.

### **Object Detection (YOLO)**
* **Sliding CNN:** Pendekatan lama, sangat lambat.
* **YOLO (You Only Look Once):** Memprediksi bounding box dan kelas dalam satu kali proses.
* Metrik evaluasi umum adalah **Mean Average Precision (mAP)**.

### **Semantic Segmentation**
* Mengklasifikasikan setiap *piksel* dalam gambar.
* Menggunakan **Fully Convolutional Networks (FCN)**.
* Menggunakan **Transposed Convolutional Layer** untuk mengembalikan resolusi spasial.


## **8. Kesimpulan**

Dalam bab ini, kita telah mempelajari:
* Evolusi CNN dari inspirasi biologis hingga arsitektur modern.
* Komponen inti: konvolusi dan pooling.
* Transfer learning sebagai praktik terbaik.
* CNN untuk klasifikasi, deteksi objek, dan segmentasi.

Di bab selanjutnya, kita akan membahas data sekuensial menggunakan RNN dan CNN.
